In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)

imputedDataset = pd.read_stata("/Users/burke/Documents/research/bpCog/mcm/mcm/data/fullyImputedDataset.dta")
imputedDataset.head()
noRecentUSInmigration = imputedDataset.loc[~imputedDataset.timeInUS.isin([1,2,3])]
earlyCrossSectionOfNonImmigrants = noRecentUSInmigration.loc[noRecentUSInmigration.year==2001]
# here we just need to throw out the very young people for comparisons
earlyCrossSectionOfNonImmigrants = earlyCrossSectionOfNonImmigrants.loc[earlyCrossSectionOfNonImmigrants.age > 18 + 2]
earlyCrossSectionOfNonImmigrants = earlyCrossSectionOfNonImmigrants.sample(n=len(earlyCrossSectionOfNonImmigrants), weights=earlyCrossSectionOfNonImmigrants.WTINT2YR, replace=True)

In [2]:
# on UCOD 1 = MI, 2=stroke, 3 = other cause of death
earlyCrossSectionOfNonImmigrants.ucod.value_counts(normalize=True, sort=False)

0    0.874492
1    0.042242
2    0.006296
3    0.076970
Name: ucod, dtype: float64

In [15]:
earlyCrossSectionOfNonImmigrants.age.describe()

count    4924.000000
mean       46.984768
std        16.756321
min        21.000000
25%        34.000000
50%        45.000000
75%        58.000000
max        85.000000
Name: age, dtype: float64

In [3]:
import os
os.chdir("/Users/burke/Documents/research/bpCog/mcm")
from mcm.population import NHANESDirectSamplePopulation
from mcm.outcome import OutcomeType

pop = NHANESDirectSamplePopulation(n=10000, year=2001)
pop.advance(10)

In [12]:
popData = pd.DataFrame({"index" : pop._people.index, 
                    "baseAge" : [person._age[0] for person in pop._people],
                    "finalAge" : [person._age[-1] for person in pop._people],
                   'mi' : [person.has_mi_during_simulation() for person in pop._people], 
                   'stroke' : [person.has_stroke_during_simulation() for person in pop._people], 
                   'miAge' : [person._outcomes[OutcomeType.MI][0][0] if len(person._outcomes[OutcomeType.MI]) > 0 else -1 for person in pop._people],
                   'strokeAge' : [person._outcomes[OutcomeType.STROKE][0][0] if len(person._outcomes[OutcomeType.STROKE]) > 0 else -1 for person in pop._people],
                   'fatalStroke' : [person.has_fatal_stroke() for person in pop._people],
                   'fatalMI' : [person.has_fatal_mi() for person in pop._people],

                   'dead' : [person.is_dead()  for person in pop._people]})

popData['age10'] = pd.qcut(popData.baseAge, 10, labels=False)
popData['nonCVDeath'] = False
popData.loc[((popData.dead) & (~popData.fatalStroke) & (~popData.fatalMI)), 'nonCVDeath'] = True


In [13]:
popData[['baseAge', 'stroke', 'mi', 'dead', 'age10', 'fatalStroke', 'fatalMI', 'nonCVDeath']].groupby('age10').mean()

,baseAge,stroke,mi,dead,fatalStroke,fatalMI,nonCVDeath
age10,,,,,,,
0,21.071222,0.011550,0.045236,0.026949,0.003850,0.006737,0.016362
1,25.869565,0.009100,0.056623,0.040445,0.002022,0.013145,0.025278
2,31.498715,0.020566,0.076264,0.042845,0.005141,0.012853,0.024850
3,37.087634,0.047712,0.137293,0.058423,0.006816,0.022395,0.029211
4,41.522337,0.059565,0.160367,0.065292,0.013746,0.018328,0.033219
5,45.881897,0.072604,0.182962,0.081317,0.015489,0.013553,0.052275
6,50.926441,0.123260,0.248509,0.126243,0.021869,0.040755,0.063618
7,56.549618,0.183206,0.324973,0.186478,0.019629,0.059978,0.106870
8,66.066800,0.334995,0.382851,0.374875,0.058824,0.066800,0.249252


In [14]:
popData[['baseAge', 'stroke', 'mi', 'dead', 'age10', 'fatalStroke', 'fatalMI', 'nonCVDeath']].mean()

baseAge        45.0453
stroke          0.1330
mi              0.2008
dead            0.1713
age10           4.4054
fatalStroke     0.0246
fatalMI         0.0319
nonCVDeath      0.1148
dtype: float64